In [28]:
"""
TensorBoardをjupyter notebookで使えるようにする．
TensorBoardとは，機械学習ワークフローの際に必要な測定値と視覚化を可能にする!!
"""
# Load the TensorBoard notebook extension
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

# Clear any logs from previous runs
!rm -rf ./logs/

In [29]:
def t_board_write(model,x,y,epoch,loss,name):#TensorBoardに計算して書き込む
    y_pred = model(x)
    CE_loss = loss(y_pred,y)
    result = torch.max(y_pred.data,dim=1).indices
    """
    y_pred.data:各サンプルに対するクラスごとの確率
    torch.max:行方向(dim=1)方向の最大値を計算する
    indices:最もスコアの高いクラスを示す
    """
    cor_pred = result.eq(y).sum()#正しく予測されたサンプル数
    accuracy = cor_pred.numpy()/len(y_pred)#予測の精度(正解率)
    writer.add_scalar("Loss/{}_Loss".format(name), CE_loss, epoch)
    writer.add_scalar("Accuracy/{}_Accuracy".format(name), accuracy, epoch)
    """
    writer.add_scalar:writerにaddしていく
    第1引数:名称
    第2引数:保存するデータ
    第3引数:反復回数
    """

In [30]:
class NetWork(nn.Module):#nn.Moduleクラスのサブクラス化(=既存のクラスを活用し新しいクラスを作成する)
    def __init__(self,input_feature,output):
        """
        input_feature:データ(特徴)の入力ユニット数
        output:出力結果への出力ユニット数
        """
        super().__init__()#コンストラクタ(=クラスを新しくした瞬間実行される関数)の上書き
        self.fc1 = nn.Linear(input_feature,output,bias=False)#線形変換する
        nn.init.xavier_normal_(self.fc1.weight)#重みをXavierの初期化
        self.fc2 = nn.Softmax(dim=1)#dim=1:行単位でsoftmaxをかけてくれる
    
    def forward(self,x):#フォワードパス(=ニューラルネットワークが初期の入力を層ごとに処理して出力に向けて送る)の定義
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [31]:
x_train = torch.load("X_train.pt")
y_train = torch.load("Y_train.pt")
x_valid = torch.load("X_valid.pt")
y_valid = torch.load("Y_valid.pt")
model = NetWork(300,4)

loss = nn.CrossEntropyLoss(reduction="mean")#クロスエントロピー損失を計算する．meanで損失の平均指定．
dataset = TensorDataset(x_train,y_train)#同じ要素数の2つのtensorを渡し，その値を得る．
dataload = DataLoader(dataset,batch_size=128,shuffle=True)

d_method = optim.SGD(params=model.parameters(),lr=10)
"""
torch.optim.SGD
params:最適化するパラメータの反復可能オブジェクト(for文が回せるやつみたいな)
lr:学習率(https://qiita.com/amateur2020/items/23704993fb191db0020c)
"""

'\ntorch.optim.SGD\nparams:最適化するパラメータの反復可能オブジェクト(for文が回せるやつみたいな)\nlr:学習率(https://qiita.com/amateur2020/items/23704993fb191db0020c)\n'

In [32]:
writer = SummaryWriter(log_dir="logs")#インスタンスを作成．log_dirでTensorBoard用のファイルを吐き出すディレクトリを指定する．
epoch = 1000

#エポック毎にバッチ処理をして損失を計算したりパラメータを更新したりする．
for ep in range (epoch):#epoch回訓練を行う
    for x,y in dataload:#128個(バッチサイズ)のデータxとラベルyを取得
        y_pred = model(x)#モデルの予測
        CE_loss = loss(y_pred,y)#クロスエントロピーの計算
        d_method.zero_grad()#計算を正しく行うために，モデル内のパラメータ勾配を初期化する．
        CE_loss.backward()#誤差逆伝搬の計算(損失を0に近づけるために必要なパラメータの勾配を効率的に計算できる)
        d_method.step()#モデルのパラメータ更新
    t_board_write(model, x_train, y_train, ep, loss, "Train")#trainデータについて損失，精度を計算しTensorBoardに書き込む．
    t_board_write(model, x_valid, y_valid, ep, loss, "Valid")#validデータについて損失，精度を計算しTensorBoardに書き込む．

torch.save(model.state_dict(), "knock75_siglelayer.pth")#モデルの保存
writer.close()

In [33]:
!%tensorboard --logdir logs/fit

zsh:fg:1: no job control in this shell.
